house (<u>***house_id*** </u>, name, <u>*host_id* </u>, description, room_type, latitude, longitud, post_time)

host (<u>***host_id***</u>, host_name, about, join_date, location, identity_verified, pic_url)

reviewer (<u>***reviewer_id***</u>, reviewer_name, about, join_date, location, identity_verified, pic_url)

price (<u>**house_id**</u>, **date**, price)

review (<u>**house_id**</u>, <u>**date**</u>, <u>***reviewer_id***</u>, review_name, host_name, score, comment, review_time, rating,reviewer_id, review_profile_path)   






详细爬取方法参考[这篇文章](https://cloud.tencent.com/developer/article/1523321)   
[各种api详细信息](https://stevesie.com/apps/airbnb-api)

### House listing api
https://www.airbnb.com/api/v2/pdp_listing_details/42471710?_format=for_rooms_show&adults=1&key=d306zoyjsyarp7ifhu67rjxn52tv0t20&    

 
### Review detail api
https://airbnb.com/api/v2/reviews?key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en&listing_id=42471710&role=guest&_format=for_p3&_limit=100

###  user(Host/Reviewer) api
https://api.airbnb.com/v2/users/13720227?key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en

### Price api

https://www.airbnb.com/api/v2/pdp_listing_booking_details?_format=for_web_dateless&_intents=p3_book_it&_interaction_type=pageload&_p3_impression_id=p3_1552520120_con%2FMRJw4xlFD6zi&_parent_request_uuid=a23f08cc-f28a-48d8-85e9-c05735e18ff9&currency=USD&force_boost_unc_priority_message_type=&guests=1&key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en&show_smart_promotion=0&checkin=2021-11-04&checkout=2021-11-09&listing_id=42471710

In [3]:
import json
import os
import time
import pickle
import requests
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from lxml import etree
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

In [36]:
# bro.get('https://www.airbnb.com/s/College-Park--MD/homes?adults=1&place_id=ChIJmUZxDR7Et4kRlW3TbeMNjOs&refinement_paths%5B%5D=%2Fhomes&checkin=2021-11-01&checkout=2021-11-04')
def getOnepage(browser, url):
    '''craw a searching page
    '''
    browser.get(url)
    time.sleep(3)
    soup = BeautifulSoup(browser.page_source,'lxml', fromEncoding = 'ISO-8859-1')
    return browser, browser.page_source, soup

# cityPagesource = bro.page_source
def get_list(soup):
    '''return a house listing from searching page
    '''
    urlList = []
    for i in soup.find_all("div", {"class": "_qlq27g"}):
        url = 'https://airbnb.com/' + i.a['href'] + '&locale=en' # force to request english version
        urlList.append(url)
    return urlList

In [42]:
bro = webdriver.Chrome()
urlList = []
for i in tqdm(range(0,10)):
    search_url = 'https://www.airbnb.com/s/College-Park--MD/homes?adults=1&place_id=ChIJmUZxDR7Et4kRlW3TbeMNjOs&refinement_paths%5B%5D=%2Fhomes&checkin=2021-11-11&checkout=2021-11-16&items_offset={}'.format(i*20)
    # print(search_url)
    _ , page_source, soup = getOnepage(bro, search_url)
    urls = get_list(soup)
    urlList.extend(urls)
bro.close()
print('the length of the url list is {}'.format(len(urlList)))


the length of the url list is 200


In [41]:
urlList
with open('urlList.data', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(urlList, filehandle)

In [7]:
with open('./urlList.data', 'rb') as filehandle:
    urlList = pickle.load(filehandle)
    
def extract_house_id(url):
    house_id = url.split('airbnb.com//rooms/')[1].split('?adults')[0]
    return house_id

In [17]:
house_id_list = [extract_house_id(i) for i in urlList]

def extract_review(json_input):
    '''iterally extract review'''
    reviewDT = pd.DataFrame()
    for rev in json_input['reviews']:
        reviewDict = {'reviewer_name':'null',
              'host_name':'null',
              'review_id':'null',
              'comments':'null', 
              'review_time':'null',
              'review_rating':'null', 
              'reviewer_id':'null'}  
        reviewDict['reviewer_name'] = rev['reviewer']['first_name'] # reviewers' first name
        reviewDict['host_name'] = rev['reviewer']['host_name'] # reviewers' first name
        reviewDict['review_id'] = str(rev['id']) # review id
        reviewDict['comments'] = rev['comments'] # comments
        reviewDict['review_time'] = rev['created_at'] # review time
        reviewDict['review_rating'] = rev['rating'] # rating
        reviewDict['reviewer_id'] = int(rev['reviewer']['id']) # reviewer id
        # reviewDict['review_profile_path'] = rev['reviewer']['profile_path'] # profile_path
        # print(reviewDict)
        reviewDT = reviewDT.append(reviewDict, ignore_index=True)
    reviewDT['reviewer_id'] = reviewDT['reviewer_id'].astype('int64')
    return reviewDT

In [37]:
reviewDt = pd.DataFrame()
error_houseid_list = []
start = time.time() 
for i in tqdm(house_id_list):
    house_review_url = 'https://www.airbnb.com/api/v2/reviews?key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en&listing_id={}&role=guest&_format=for_p3&_limit=100'.format(i)
    time.sleep(1)
    response = requests.get(house_review_url)
    json_input = response.json()
    try:
        dt1 = extract_review(json_input)
    except:
        error_houseid_list.append(i)
    reviewDt = reviewDt.append(dt1, ignore_index = True)
print('data has {} rows and {} columns'.format(reviewDt.shape[0], reviewDt.shape[1]))
print('Appending data costs %.2f seconds'%(time.time() - start))
print('{} data has errors'.format(len(error_houseid_list)))


data has 7130 rows and 7 columns
Reading data costs 255.09 seconds
13 data has errors


In [38]:
reviewDt

,comments,host_name,review_id,review_rating,review_time,reviewer_id,reviewer_name
0,Excellent host and room. Exceptionally clea...,Bonnie,476038640910050069,5.0,2021-10-18T19:23:48Z,147939426,Bonnie
1,"Great home, Warren was a great host. Definitel...",Jessica,475361095456175886,5.0,2021-10-17T20:57:38Z,19926821,Jessica
2,The Best of Greenbelt!,Kaitlyn,473800873862654389,5.0,2021-10-15T17:17:45Z,174454317,Kaitlyn
3,You will be quite pleased! Warren provides a ...,Sheira,470191061094878092,5.0,2021-10-10T17:45:42Z,63268026,Sheira
4,"Clean townhome, and clean room; Warren is frie...",Jay,449903203049691759,5.0,2021-09-12T17:57:21Z,273090517,Jay
...,...,...,...,...,...,...,...
7125,"I recommend this place overall. Very clean, an...",Chris,567122911,5.0,2019-11-21T21:55:35Z,89582884,Chris
7126,"Easy check in, super clean, quiet neighborhood...",Chris,564298709,5.0,2019-11-15T18:49:45Z,126613729,Chris
7127,Good location and price.,Doug,560727554,5.0,2019-11-07T21:34:27Z,14966529,Doug
7128,Excellent accommodations. Communication was t...,Alissa,560071093,5.0,2019-11-05T20:06:19Z,57923452,Alissa


In [40]:
def extract_json_listing(listing_json):
    '''extract listing json
    '''
    listingDict = {'house_id':'null',
              'host_name':'null',
              'host_id':'null',
              'title':'null', 
              'address':'null',
              'latitude':'null', 
              'longtitude':'null', 
              'checkin_rating':'null',
              'clean_rating':'null',
              'comm_rating':'null',
              'satis_overall':'null',
              'bathroom_type':'null',
              'bed_type':'null',
              'bedroom_type':'null',
              'guest_type':'null',
    }
    
    listingDict['house_id'] = listing_json['pdp_listing_detail']['id']
    listingDict['host_name'] = listing_json['pdp_listing_detail']['primary_host']['host_name']
    listingDict['host_id'] = listing_json['pdp_listing_detail']['primary_host']['id']
    listingDict['title'] = listing_json['pdp_listing_detail']['name'] # title
    listingDict['address'] = listing_json['pdp_listing_detail']['p3_summary_address'] # address 
    listingDict['latitude'] = listing_json['pdp_listing_detail']['p3_event_data_logging']['listing_lat'] # lat
    listingDict['longtitude'] = listing_json['pdp_listing_detail']['p3_event_data_logging']['listing_lng'] # lng
    listingDict['checkin_rating'] = listing_json['pdp_listing_detail']['p3_event_data_logging']['checkin_rating'] 
    listingDict['clean_rating'] = listing_json['pdp_listing_detail']['p3_event_data_logging']['cleanliness_rating'] 
    listingDict['comm_rating'] = listing_json['pdp_listing_detail']['p3_event_data_logging']['communication_rating'] 
    listingDict['satis_overall'] = listing_json['pdp_listing_detail']['p3_event_data_logging']['guest_satisfaction_overall']
    # JSON.pdp_listing_detail.bathroom_label
    listingDict['bathroom_type'] = listing_json['pdp_listing_detail']['bathroom_label']
    listingDict['bed_type'] = listing_json['pdp_listing_detail']['bed_label']
    listingDict['bedroom_type'] = listing_json['pdp_listing_detail']['bedroom_label']
    listingDict['guest_type'] = listing_json['pdp_listing_detail']['guest_label']
    return listingDict


listingDt = pd.DataFrame()
error_houseling_list = []
start = time.time() 
for i in tqdm(house_id_list):
    listing_url = 'https://www.airbnb.com/api/v2/pdp_listing_details/{}?_format=for_rooms_show&adults=1&key=d306zoyjsyarp7ifhu67rjxn52tv0t20&'.format(i)
    time.sleep(1)
    response = requests.get(listing_url)
    json_input = response.json()
    try:
        dt1 = extract_json_listing(json_input)
    except:
        error_houseling_list.append(i)
    listingDt = listingDt.append(dt1, ignore_index = True)
print('data has {} rows and {} columns'.format(listingDt.shape[0], listingDt.shape[1]))
print('Appending data costs %.2f seconds'%(time.time() - start))
print('{} data has errors'.format(len(error_houseling_list)))


data has 155 rows and 15 columns
Appending data costs 222.15 seconds
2 data has errors


In [44]:
url = 'https://api.airbnb.com/v2/users/13720227?key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en'
response = requests.get(url)
user_json = response.json()
user_json

# host (host_id, host_name, about, join_date, location, identity_verified, pic_url)
# reviewer (reviewer_id, reviewer_name, about, join_date, location, identity_verified, pic_url)
def extract_user(user_json, reviewer=True):
    '''extract reviewer/host and host information
    '''
    userDict = {
              'first_name':'null',
              'join_date':'null',
              'about':'null',
              'identity_verified':'null', 
              'picture_url':'null',
              'created_at':'null', 
              'location':'null'
    }
    if reviewer:
        userDict['reviewer_id'] = user_json['user']['id']
    else:
        userDict['host_id'] = user_json['user']['id']
    userDict['first_name'] = user_json['user']['first_name']
    userDict['join_date'] = user_json['user']['created_at']
    userDict['about'] = user_json['user']['about']
    userDict['identity_verified'] = user_json['user']['identity_verified']
    userDict['picture_url'] = user_json['user']['picture_url']
    userDict['created_at'] = user_json['user']['created_at']
    userDict['location'] = user_json['user']['location']
    return userDict
extract_user(user_json)

{'reviewer_id': 13720227,
 'first_name': 'Brad',
 'join_date': '2014-03-31T06:01:53Z',
 'about': "I've lived in The District for nearly 34 years.  I love this great city and all it offers.  Feel free to ask me for recommendations or any other information about DC.  I've lived in this house since 2001 and have enjoyed hosting people from all over the world.",
 'identity_verified': True,
 'picture_url': 'https://a0.muscache.com/im/pictures/user/f4d9817e-6615-4d56-ab4b-4479da564cd6.jpg?aki_policy=profile_x_medium',
 'created_at': '2014-03-31T06:01:53Z',
 'location': 'Washington, DC'}

In [10]:
a = {'a':123,'b':2342,'c':4534}
a['d'] = 342342
a

{'a': 123, 'b': 2342, 'c': 4534, 'd': 342342}

In [14]:
# Making a get request
#house_review_json = 'https://www.airbnb.com/api/v2/reviews?key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en&listing_id=42471710&role=guest&_format=for_p3&_limit=100'
#response = requests.get(house_review_json)
# print json content
#print()
#house_review_json = 'https://www.airbnb.com/api/v2/reviews?key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en&listing_id=16465485&role=guest&_format=for_p3&_limit=100'
#response = requests.get(house_review_json)
#response.json()

## response.json()['reviews'][3]['comments']
## listing-42471710 > div > div._ni9axhe > div > div._e296pg > div > div > div > a
## soup = BeautifulSoup(cityPagesource,'lxml', fromEncoding = 'ISO-8859-1')
## url.split('airbnb.com//rooms/')[1].split('?adults')[0] # 爬取room_id
## 
## soup.title.string.split('-')[0] # 爬取title
## soup.title.string.split('-')[1] # 爬取地点
## soup.title.string.split('-')[1].split(',')[-2].strip() # 州
## soup.title.string.split('-')[1].split(',')[-1].strip() # 国
## soup.title.string.split('-')[1].split(',')[-3].split('in')[-1].strip() # city
## 
## soup.find_all("a", {"class": "_105023be"})[0]['href'] # host id
## 
## list1 = []
## for i in soup.find_all("ol", {"class": "_194e2vt2"}):
##     list1.extend([i.string for i in i.find_all('span')])  # 
##     #url =  i.a['href'] # 强行改为英文网页
##     #print(url)
## list1
## soup.find_all("a", {"class": "_105023be"})[0]['href']
## Host information
## for i in soup.find_all("span", {"class": "_1ax9t0a"}):
##   print(i.get_text())
## soup.find_all("h1", {"class": "_14i3z6h"})[1].get_text() # host name
## soup.find_all("div", {"class": "_u5okn1"})[0].get_text() # join date
## soup.find_all("span", {"class": "_1ax9t0a"})[0].get_text() # reviews number
## soup.find_all("span", {"class": "_1ax9t0a"})[1].get_text() # Identity Verification
## soup.find_all("span", {"class": "_1ax9t0a"})[6].get_text() # address 
## soup.find_all("span", {"class": "_1ax9t0a"})[4].get_text()
## soup.find_all("div", {"class": "_u5okn1"})[0].get_text()
## soup.find_all("div", {"class": "_u5okn1"})[0].get_text()
## response.json()['reviews'][4]['comments']